In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as web
import pandas as pd
import math

In [2]:
data = web.DataReader('EURUSD=X', data_source='yahoo', start='2011-12-31', end='2019-12-31') 

In [3]:
data_close = data.filter(['Close'])
dataset = data_close.values

In [4]:
train_len = math.ceil( len(dataset) *.8)

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
training_data = scaler.fit_transform(dataset[:train_len , : ])

In [6]:
x_train =[]
y_train =[]
for i in range(60, len(training_data)):
  x_train.append(training_data[i-60:i,0])
  y_train.append(training_data[i,0])

In [7]:
x_train,y_train = np.array(x_train), np.array(y_train)
x_train= np.reshape(x_train,( x_train.shape[0] ,x_train.shape[1] ,1))

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM( 50, return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.save('lstm_model.h5')

In [9]:
model.fit(x_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
51/51 [==============================] - 4s 33ms/step - loss: 0.0418
Epoch 2/100
51/51 [==============================] - 2s 31ms/step - loss: 0.0042
Epoch 3/100
51/51 [==============================] - 2s 32ms/step - loss: 0.0036
Epoch 4/100
51/51 [==============================] - 2s 31ms/step - loss: 0.0029
Epoch 5/100
51/51 [==============================] - 2s 31ms/step - loss: 0.0028
Epoch 6/100
51/51 [==============================] - 2s 29ms/step - loss: 0.0026
Epoch 7/100
51/51 [==============================] - 2s 30ms/step - loss: 0.0027
Epoch 8/100
51/51 [==============================] - 2s 31ms/step - loss: 0.0025
Epoch 9/100
51/51 [==============================] - 2s 32ms/step - loss: 0.0026
Epoch 10/100
51/51 [==============================] - 2s 31ms/step - loss: 0.0021
Epoch 11/100
51/51 [==============================] - 2s 31ms/step - loss: 0.0022: 0s - loss: 0.
Epoch 12/100
51/51 [==============================] - 2s 29ms/step - loss: 0.0019
Epoch 13/1

In [10]:
#remaining 20%
test_data = scaler.transform(dataset[train_len - 60: , : ])
x_test = []
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])
  
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))
x_test.shape

(416, 60, 1)

In [11]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)# importance of transforme only
predictions.shape

(416, 1)

In [12]:
model.save('lstm_model.h5')